In [17]:
import os, io
import pandas as pd
import pysam
import pickle
import pybedtools
pybedtools.helpers.set_tempdir('/data/projects/temp')
from IPython.display import display, HTML

In [12]:
df_acceptor = pd.read_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Acceptor_RAW_Combined.tsv", sep="\t")
df_acceptor['start'] = df_acceptor['coordinates']-40
df_acceptor['end'] = df_acceptor['coordinates']+39

In [3]:
df_acceptor

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,acceptor_40BP,start,end
0,0,ENST00000003583.12,ENSE00003692979.1,protein_coding,-,24401457,TCAACAGTGACTTCTTACGCAGTTCCTACTGCAGGCAGCACCAGCA...,chr1,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...,24401417,24401496
1,1,ENST00000003583.12,ENSE00003638911.1,protein_coding,-,24384004,TTCCAGCCCCACCACTTGGTAACTGAGACGAAGATCGGGCAAGTCA...,chr1,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...,24383964,24384043
2,2,ENST00000003583.12,ENSE00003688858.1,protein_coding,-,24379824,AAGCGAGTACAGGTCTGAACTTGCGTTTTCCAAGGGAAGATCCTCA...,chr1,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...,24379784,24379863
3,3,ENST00000003583.12,ENSE00003631141.1,protein_coding,-,24373811,AAACTCATAGACCACTTGAGCTGGGATGACAGGGAAGCCACAGCCA...,chr1,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...,24373771,24373850
4,4,ENST00000003583.12,ENSE00003516207.1,protein_coding,-,24369840,AGCTAAGTGTGTGCTTCCCGTGCCATCTGTTCTCTGCATGGTGAGG...,chr1,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...,24369800,24369879
...,...,...,...,...,...,...,...,...,...,...,...
1255761,1261995,ENST00000684226.1,ENSE00003767626.1,non_stop_decay,-,13297849,AATGTCTCGCATATTGTGACTGCTCTTTACGTTAGGTTTTTTTTTA...,chrY,ACTTTGGGTTACTTTATGACTGTTGCTTTTTTCCTTCTAGCCTGCC...,13297809,13297888
1255762,1261996,ENST00000684226.1,ENSE00003761778.1,non_stop_decay,-,13260405,GTCAAATGTCAGTTATTCTCAACTTTAAATATTTTAATATTGTCAT...,chrY,TTCAGTAGTTACTCTCACAAATAGTTTTTTCCCACAATAGGTATTG...,13260365,13260444
1255763,1261997,ENST00000684226.1,ENSE00001713248.1,non_stop_decay,-,13251188,TTTCTCTGGGTGTGCTACAAAACAAAACCTAGTGTCTCAAACTCAC...,chrY,ACTGTCCAATACTTTTTGTTTATTTATTTATTTAATATAGGTGGAG...,13251148,13251227
1255764,1261998,ENST00000684226.1,ENSE00003917271.1,non_stop_decay,-,13234827,AGCTTGCACTGACCTGCAGGTGTTCCTTGATATATACATCCTGGGT...,chrY,TCCTGCAAACCCCAATGTGAACTTGTGGTGCCTTTTCCAGGCCTCC...,13234787,13234866


In [4]:
df_acceptor_bed = df_acceptor[['chr_name', 'start',	'end', 'strand','transcript_id','exon_id','transcript_type']]

In [5]:
df_acceptor_bed

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,24401417,24401496,-,ENST00000003583.12,ENSE00003692979.1,protein_coding
1,chr1,24383964,24384043,-,ENST00000003583.12,ENSE00003638911.1,protein_coding
2,chr1,24379784,24379863,-,ENST00000003583.12,ENSE00003688858.1,protein_coding
3,chr1,24373771,24373850,-,ENST00000003583.12,ENSE00003631141.1,protein_coding
4,chr1,24369800,24369879,-,ENST00000003583.12,ENSE00003516207.1,protein_coding
...,...,...,...,...,...,...,...
1255761,chrY,13297809,13297888,-,ENST00000684226.1,ENSE00003767626.1,non_stop_decay
1255762,chrY,13260365,13260444,-,ENST00000684226.1,ENSE00003761778.1,non_stop_decay
1255763,chrY,13251148,13251227,-,ENST00000684226.1,ENSE00001713248.1,non_stop_decay
1255764,chrY,13234787,13234866,-,ENST00000684226.1,ENSE00003917271.1,non_stop_decay


In [15]:
# Group by 'start' and 'end' and aggregate other columns
df_acceptor_unique_bed = df_acceptor_bed.groupby(['chr_name','start', 'end']).agg(lambda x: ", ".join(x.unique())).reset_index()
df_acceptor_unique_bed

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12572,12651,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001758273.2, ENSE00003582793.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12934,13013,+,ENST00000450305.2,ENSE00001799933.2,transcribed_unprocessed_pseudogene
2,chr1,13180,13259,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001746346.2, ENSE00002312635.1","transcribed_unprocessed_pseudogene, processed_..."
3,chr1,13412,13491,+,ENST00000450305.2,ENSE00001863096.1,transcribed_unprocessed_pseudogene
4,chr1,14999,15078,-,ENST00000488147.1,ENSE00001935574.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
295391,chrY,57211720,57211799,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00001956598.1, ENSE00001416295.3, ENSE0000...","retained_intron, protein_coding, processed_tra..."
295392,chrY,57213086,57213165,-,ENST00000507418.6_PAR_Y,ENSE00002023900.1,unprocessed_pseudogene
295393,chrY,57213318,57213397,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
295394,chrY,57213563,57213642,-,ENST00000507418.6_PAR_Y,ENSE00002021169.1,unprocessed_pseudogene


In [16]:
df_acceptor_unique_bed[df_acceptor_unique_bed['start']==70051266]

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
157101,chr2,70051266,70051345,-,"ENST00000366234.7, ENST00000411429.5, ENST0000...","ENSE00001639513.1, ENSE00001644011.1, ENSE0000...",lncRNA


In [23]:
df_acceptor_unique_bed.to_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Unique_Acceptor_RAW_Combined_BED.tsv", sep="\t", index=False)

In [19]:
with open("/data/projects/VCF_files/BRCA/Postprocessed_files/vcf_data.pkl", "rb") as file:
    loaded_dictionary = pickle.load(file)

In [21]:
acceptor_bed = pybedtools.BedTool.from_dataframe(df_acceptor_unique_bed)
vcf_acceptor_data = {}
index = 0
for key, value in loaded_dictionary.items():
    #display(value)
    vcf_bed = pybedtools.BedTool.from_dataframe(value)
    if((index%200)==0):
        print(index, key)
    index= index+1
    # Use intersect function
    intersect_vcf_acceptor = acceptor_bed.intersect(vcf_bed, wa=True, wb=True)
    column_list  = df_acceptor_bed.columns.to_list() + value.columns.to_list()
    #print(column_list)
    df_intersection = intersect_vcf_acceptor.to_dataframe(names=column_list)
    #display(df_intersection)
    #input()
    vcf_acceptor_data[key] = df_intersection

0 c5bbc8ae-3efe-43a7-9613-ce7f967b34a7.wxs.mutect2.raw_somatic_mutation.vcf.gz
200 TCGA_BRCA.8e9c3ea3-2c4d-4d2e-add4-e8992574b45a.wxs.Pindel.somatic_annotation.vcf.gz
400 3593447c-3a0d-485b-9743-3146ccc5081a.wxs.varscan2.raw_somatic_mutation.vcf.gz
600 94068b27-d41c-4db0-8f73-3e0db65aeb20.wxs.varscan2.raw_somatic_mutation.vcf.gz
800 52b3dee4-21ae-4131-8a1d-b1a5ebc1e3c7.wxs.varscan2.raw_somatic_mutation.vcf.gz
1000 TCGA_BRCA.7e5e9662-c941-4aeb-843e-0db3946f7d98.wxs.VarScan2.somatic_annotation.vcf.gz
1200 f7116df5-11e6-4fcb-83a0-bff3f201a836.wxs.mutect2.raw_somatic_mutation.vcf.gz
1400 TCGA_BRCA.d83d5baf-04ce-459a-8131-cae34461baa2.wxs.VarScan2.somatic_annotation.vcf.gz
1600 7fa0da5a-951b-46e6-9518-b14d4a6ee85b.wxs.pindel.raw_somatic_mutation.vcf.gz
1800 4eecd5a8-0139-49e3-aa2b-5b6ffd1eed03.wxs.varscan2.raw_somatic_mutation.vcf.gz
2000 TCGA_BRCA.45acd765-57b5-4065-9593-79261328ce30.wxs.VarScan2.somatic_annotation.vcf.gz
2200 TCGA_BRCA.f97bf6d4-5d3e-480b-aeaa-92e81e41161b.wxs.MuTect2.soma

In [22]:
with open("/data/projects/VCF_files/BRCA/Postprocessed_files/intersect_acceptor_vcf_data.pkl", "wb") as file:
    pickle.dump(vcf_acceptor_data, file)